A collection of docs on different web-pages

In [1]:
docs = [
    '''About us. We deliver Artificial Intelligence & Machine Learning
       solutions to solve business challenges.''',
    '''Contact information. Email [martin davtyan at filament dot ai]
       if you have any questions''',
    '''Filament Chat. A framework for building and maintaining a scalable
       chatbot capability''',
]


In [2]:
import string
import nltk
from nltk.tokenize import TreebankWordTokenizer

REMOVE_PUNCTUATION_TABLE = str.maketrans({x: None for x in string.punctuation})
TOKENIZER = TreebankWordTokenizer()

example_doc = docs[1]
example_doc_tokenized = TOKENIZER.tokenize(
        example_doc.translate(REMOVE_PUNCTUATION_TABLE)
        )
example_doc_tokenized

['Contact',
 'information',
 'Email',
 'martin',
 'davtyan',
 'at',
 'filament',
 'dot',
 'ai',
 'if',
 'you',
 'have',
 'any',
 'questions']

We can't find any overlapping words between the doc and the user query because the user query might not be capitalised or it may be plural. To solve this issue, we can use stemming which is basically stripping the words of their formatting.


In [3]:
from nltk.stem import PorterStemmer
STEMMER = PorterStemmer()

example_doc_stemmed = [STEMMER.stem(token) for token in example_doc_tokenized]
example_doc_stemmed

['contact',
 'inform',
 'email',
 'martin',
 'davtyan',
 'at',
 'filament',
 'dot',
 'ai',
 'if',
 'you',
 'have',
 'ani',
 'question']

In [4]:
def tokenize_and_stem(s):
    return [STEMMER.stem(t) for t
            in TOKENIZER.tokenize(s.translate(REMOVE_PUNCTUATION_TABLE))]
query = 'contacts'
tokenize_and_stem(query)

['contact']

Vector-Space Model

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
VECTORIZER = TfidfVectorizer(tokenizer=tokenize_and_stem, stop_words='english')
VECTORIZER.fit(docs)
sorted_vocab = sorted(VECTORIZER.vocabulary_.items())
print(sorted_vocab)

[('ai', 0), ('ani', 1), ('artifici', 2), ('build', 3), ('busi', 4), ('capabl', 5), ('challeng', 6), ('chat', 7), ('chatbot', 8), ('contact', 9), ('davtyan', 10), ('deliv', 11), ('dot', 12), ('email', 13), ('filament', 14), ('framework', 15), ('inform', 16), ('intellig', 17), ('learn', 18), ('machin', 19), ('maintain', 20), ('martin', 21), ('question', 22), ('scalabl', 23), ('solut', 24), ('solv', 25)]


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', '

A matrix of TF-Idf scores of all the documents given the query. So, whihc docs match the given query the most. Basically a similarity score btween each doc and the query.

In [8]:
query = 'contact email is needed to chat with the client'
query_vector = VECTORIZER.transform([query]).todense()
query_vector

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.57735027, 0.        , 0.57735027,
         0.        , 0.        , 0.        , 0.57735027, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ]])

In [9]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

doc_vectors = VECTORIZER.transform(docs).toarray()
# Convert query_vector to a numpy array
query_vector = np.asarray(query_vector)
similarity = cosine_similarity(query_vector, doc_vectors)
similarity

array([[0.        , 0.37309798, 0.2097252 ]])

In [10]:
ranks = (-similarity).argsort(axis=None)
ranks

array([1, 2, 0])

In [11]:
most_relevant_doc = docs[ranks[0]]
most_relevant_doc

'Contact information. Email [martin davtyan at filament dot ai]\n       if you have any questions'

Our system is returning a chatbot related page given a query that has chatbot word in it.
Dict feedback has queries as keys and (document index, feedback value) as its values.

In [12]:
feedback = {
        'who makes chatbots': [(2, 0.), (0, 1.), (1, 1.), (0, 1.)],
        'about page': [(0, 1.)]
}
similarity = cosine_similarity(VECTORIZER.transform(['who makes chatbots']), doc_vectors)
ranks = (-similarity).argsort(axis=None)
docs[ranks[0]]

'Filament Chat. A framework for building and maintaining a scalable\n       chatbot capability'

Feedback data includes information about which documents were deemed relevant or irrelevant for previous queries. This data can be used to improve the scoring of documents for new queries.

Positive feedback feature- number of times a document has appeared relevant for a given query.

Negative feedback feature - number of time sa given documnet has appeared irrelevant.

Increase the score for documents with positive feedback and decrease it for those with negative feedback. If there's no feedback, the features are zero and don't impact the score.

In [15]:
import numpy as np
import string
from sklearn.feature_extraction.text import TfidfVectorizer

query = 'who is making chatbots information'
feedback_queries = list(feedback.keys())

similarity = cosine_similarity(VECTORIZER.transform([query]),
                               VECTORIZER.transform(feedback_queries))
similarity



max_idx = np.argmax(similarity)
feedback_queries[max_idx]

'who makes chatbots'

In [16]:
pos_feedback_doc_idx = [idx for idx, feedback_value
                        in feedback[feedback_queries[max_idx]]
                        if feedback_value == 1.]
pos_feedback_doc_idx

[0, 1, 0]

[0, 1, 0]

In [17]:
from collections import Counter

counts = Counter(pos_feedback_doc_idx)
counts

Counter({0: 2, 1: 1})

pos_feedback_proportions = {
        doc_idx: count / sum(counts.values()) for doc_idx, count in counts.items()
}
pos_feedback_proportions

{0: 0.6666666666666666, 1: 0.3333333333333333}

In [18]:
nn_similarity = np.max(similarity)
pos_feedback_feature = [nn_similarity * pos_feedback_proportions.get(idx, 0.)
                        for idx, _ in enumerate(docs)]
pos_feedback_feature

[0.4714045207910317, 0.23570226039551584, 0.0]

# Handling unseen queries

if the given query hasn't apperaed before in the search, it becomes difficult to determine the relevance of documents accurately.
One solution can be using the nearest neighbour approach, i.e, looking for a previously encountered query that is closest to the new query in terms of similarity. By using the method of TF-IDF to measure the similarity between the new and the past queries.

Or use cosine similarity.

In [23]:
class Scorer():
    """ Scores documents for a search query based on tf-idf
        similarity and relevance feedback

    """
    def __init__(self, docs):
        """ Initialize a scorer with a collection of documents, fit a
            vectorizer and list feature functions

        """
        self.docs = docs

        self.vectorizer = TfidfVectorizer(tokenizer=tokenize_and_stem,
                                          stop_words='english')
        self.doc_tfidf = self.vectorizer.fit_transform(docs)

        self.features = [
            self._feature_tfidf,
            self._feature_positive_feedback,
        ]
        self.feature_weights = [
            1.,
            2.,
        ]

        self.feedback = {}

    def score(self, query):
        """ Generic scoring function: for a query output a numpy array
            of scores aligned with a document list we initialized the
            scorer with

        """
        feature_vectors = [feature(query) for feature
                           in self.features]

        feature_vectors_weighted = [feature * weight for feature, weight
                                    in zip(feature_vectors, self.feature_weights)]
        return np.sum(feature_vectors_weighted, axis=0)

    def learn_feedback(self, feedback_dict):
        """ Learn feedback in a form of `query` -> (doc index, feedback value).
            In real life it would be an incremental procedure updating the
            feedback object.

        """
        self.feedback = feedback_dict

    def _feature_tfidf(self, query):
        """ TF-IDF feature. Return a numpy array of cosine similarities
            between TF-IDF vectors of documents and the query

        """
        query_vector = VECTORIZER.transform([query])
        similarity = cosine_similarity(query_vector, self.doc_tfidf)
        return similarity.ravel()

    def _feature_positive_feedback(self, query):
        """ Positive feedback feature. Search the feedback dict for a query
            similar to the given one, then assign documents positive values
            if there is positive feedback about them.

        """
        if not self.feedback:
            return np.zeros(len(self.docs))

        feedback_queries = list(self.feedback.keys())
        similarity = cosine_similarity(self.vectorizer.transform([query]),
                                       self.vectorizer.transform(feedback_queries))
        nn_similarity = np.max(similarity)

        nn_idx = np.argmax(similarity)
        pos_feedback_doc_idx = [idx for idx, feedback_value in
                                self.feedback[feedback_queries[nn_idx]]
                                if feedback_value == 1.]

        feature_values = {
                doc_idx: nn_similarity * count / sum(counts.values())
                for doc_idx, count in Counter(pos_feedback_doc_idx).items()
        }
        return np.array([feature_values.get(doc_idx, 0.)
                         for doc_idx, _ in enumerate(self.docs)])

In [24]:
scorer = Scorer(docs)
query


'who is making chatbots information'

In [25]:
scorer.score(query)

array([0.        , 0.22847492, 0.25685987])

In [26]:
docs[scorer.score(query).argmax()]

'Filament Chat. A framework for building and maintaining a scalable\n       chatbot capability'

In [27]:
scorer.learn_feedback(feedback)
scorer.score(query)

array([0.94280904, 0.69987944, 0.25685987])

In [28]:
docs[scorer.score(query).argmax()]

'About us. We deliver Artificial Intelligence & Machine Learning\n       solutions to solve business challenges.'

In [29]:
scorer.feature_weights = [0.6, 0.4]
scorer.score(query)


array([0.18856181, 0.23136585, 0.15411592])

In [30]:
docs[scorer.score(query).argmax()]

'Contact information. Email [martin davtyan at filament dot ai]\n       if you have any questions'